In [1]:
1+1

2

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [4]:
%pwd

'd:\\Shoeb\\MLOPS\\datascience\\research'

In [5]:
import os

In [6]:
os.chdir("../")

In [7]:
%pwd

'd:\\Shoeb\\MLOPS\\datascience'

In [16]:
!pip install pyyaml

  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl (161 kB)


In [7]:
import yaml

In [19]:
!pip install joblib

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


In [21]:
!pip install ensure

  Using cached ensure-1.0.4-py3-none-any.whl.metadata (10 kB)
Using cached ensure-1.0.4-py3-none-any.whl (15 kB)


In [30]:
! pip uninstall box

^C


In [31]:
! pip install box

In [11]:
pip install python-box


  Using cached python_box-7.2.0-cp310-cp310-win_amd64.whl.metadata (8.0 kB)
Using cached python_box-7.2.0-cp310-cp310-win_amd64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip show python-box


Name: python-box
Version: 7.2.0
Summary: Advanced Python dictionaries with dot notation access
Home-page: https://github.com/cdgriffith/Box
Author: Chris Griffith
Author-email: chris@cdgriffith.com
License: MIT
Location: d:\shoeb\mlops\datascience\.venv\lib\site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install --upgrade ipython stack_data


  Using cached executing-2.1.0-py2.py3-none-any.whl.metadata (8.9 kB)
Using cached executing-2.1.0-py2.py3-none-any.whl (25 kB)
  Attempting uninstall: executing
    Found existing installation: executing 0.8.2
    Uninstalling executing-0.8.2:
      Successfully uninstalled executing-0.8.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
box 0.1.5 requires executing==0.8.2, but you have executing 2.1.0 which is incompatible.


In [2]:
from box import ConfigBox


In [9]:
pip install --upgrade stack_data


Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install --upgrade ipython


In [8]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [23]:
class ConfigurationManager:
    def __init__ (self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(

            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config



In [24]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [25]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [26]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-04 23:48:49,692: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-04 23:48:49,694: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-04 23:48:49,695: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-04 23:48:49,696: INFO: common: created directory at: artifacts]
[2024-12-04 23:48:49,697: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-04 23:48:50,471: INFO: 251335794: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0B5C:35A183:163F5A:1B6E70:67509D09
Accept-Ranges: bytes
Date: Wed, 04 